## Chapter 07. 假设与推断
注：本章涉及统计学的知识，等学习完统计学后在仔细阅读。
        －－ by alex 2017.12.28

具备以上统计学和概率理论知识以后，我们接着该做什么呢？数据科学的科学部分，乃是不断针对我们的数据和生成数据的机制建立假设和检验假设。

### 7.1 统计假设检验
作为数据科学家，我们常常需要检验某个假设是否成立。诸如，“这枚硬币是否均匀”、“如果人们点开某个突然弹出的小广告，广告的关闭按钮又小又难找，那么大家更倾向于离开这个页面，压根不会阅读”等可以被翻译成统计数据的断言。在各种各样的假设之下，这些统计数据可以理解为从某种已知分布中抽取的随机变量观测值，这可以让我们对这些假设是否成立做出论断。

典型的步骤是这样的，首先我们有一个零假设$H_0$， 它代表一个默认的立场，而替代假设$H_1$代表我们希望与零假设对比的立场。我们通过统计来决定我们是否可以拒绝$H_0$，即判断它是否错误。通过举例能更直观地说明这个过程。

### 7.2 案例 ： 掷硬币
假设有一枚硬币，我们试图判断它是否均匀，即任何一面朝上的可能性是否相等。首先，假设硬币落地后正面朝上的概率为$p$，所以我们的零假设为硬币均匀，即$p=0.5$。我们要对比替代假设$p \ne 0.5$来检验这个假设。

具体来说，首先掷硬币$n$次，将出现正面朝上的次数记为$X$。每次掷硬币都是一次伯努利试验，意味着$X$是二项式随机变量$Binomial(n,p)$，（如第6章中所讲到的）可以用正态分布来拟合：

In [1]:
# 导入本章用到的库
import math, random

# 根据n(实验次数)和p(概率)计算正态分布的mu和sigma
def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

只要一个随机变量服从正态分布，我们就可以用$normal\_cdf$来计算出一个实现数值位于(或不在)某个特定区间的概率：

In [2]:
# 正态分布的累积分布函数
def normal_cdf(x, mu=0, sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

# 正态cdf是一个变量在一个阈值以下的概率
normal_probability_below = normal_cdf

# 如果它不在阈值以下,就在阈值之上
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# 如果它小于hi但不比lo小,那么它在区间之内
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# 如果不在区间之内,那么就在区间之外
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

或者反过来，找出非尾区域，或者找出均值两边的（对称）区域，这个区域恰好对应特定比例的可能性。比如，如果我们需要找出以均值为中心、覆盖$60\%$可能性的区间，那我们需要找到两个截点，使上尾和下尾各覆盖$20\%$的可能性（给中间留出$60\%$）：

In [3]:
# 对normal_cdf取逆, 从而可以求出特定的概率的相应值
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""
    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)

    low_z, low_p = -10.0, 0            # normal_cdf(-10) is (very close to) 0
    hi_z,  hi_p  =  10.0, 1            # normal_cdf(10)  is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break
    return mid_z

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return lower_bound, upper_bound

具体来讲，首先我们选择掷硬币$n=1000$次。如果关于均匀的原假设是正确的，那么$X$近似服从正态分布，均值为$500$，标准差为$15.8$：

In [4]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
mu_0, sigma_0

(500.0, 15.811388300841896)

我们需要对`显著性（signiﬁcance）`下定义——我们有多大的可能性犯第1类错误（“容 错”）。在这种情况下，我们拒绝了原假设$H_0$，但实际上原假设是正确的。出于历史上的某些原因，可能性的大小通常设定为$5\%$或者$1\%$。 本书在此选择$5\%$。

考虑这样的检验——如果$X$落在以下区间以外，就拒绝原假设$H_0$。假设$p$实际上等于$0.5$（即$H_0$成立），那么我们有$5\%$ 的可能观测到$X$落在区间之外，这正是我们想要的显著性。 换句话说，如果$H_0$为真，那么$20$次检验中大约有$19$次会得出正确的结果。下面的测试表明，如果允许$5\%$的误差，那么1000次测试中，正面的次数不能少于$469$次，不能多于$530$次，否则就拒绝$H_0$：

In [ ]:
normal_two_sided_bounds(0.95, mu_0, sigma_0)

我们常常对检验的`势（power）`有兴趣，它指的是不犯第2类错误的概率。 第2类错误指原假设$H_0$是错的，但我们的检验结果没有拒绝原假设（即“纳伪”）。为了衡量统计的势，我们需要精确衡量$H_0$是错的意味着什么。（仅仅知道$p$不是$0.5$不足以为$X$的分布提供足够的信息。）具体来说，假如$p$实际上是$0.55$， 那么掷硬币的结果会稍微多偏向正面朝上。

在这种情形下，我们这样计算检验的势：

In [ ]:
# 基于假设p是0.5时95%的边界 
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
lo, hi

In [ ]:
normal_probability_between(lo, hi, mu_0, sigma_0)

In [ ]:
# 基于p = 0.55的真实mu和sigma
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
mu_1, sigma_1

In [ ]:
# 第2类错误意味着我们没有拒绝原假设 
# 这会在X仍然在最初的区间时发生 
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1) 
type_2_probability

In [ ]:
power = 1 - type_2_probability
power

如果我们把原假设变为掷硬币的结果不会偏重于正面朝上，即$P \le 0.5$，在这种情形下，我们使用单边检验。如果$X$远大于$50$，我们就拒绝原假设，如果$X$小于$50$，就不拒绝原假设。因此，显著性为$5\%$的检验需要使用$normal\_probability\_below$来找出小于$95\%$的概率对应的截点：

In [ ]:
hi = normal_upper_bound(0.95, mu_0, sigma_0) 
# 是526 (< 531, 因为我们在上尾需要更多的概率)
hi

In [ ]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1) 
power = 1 - type_2_probability # 0.936
power

这是更有效的检验。如果$X$小于$469$， 我们就不再拒绝$H_0$（如果$H_1$为真，这不太可能发生），当$X$在$526$和$531$之间时则拒绝$H_0$（如果$H_1$为真，这很有可能发生）。